In [1]:
import netrustpy3
import networkx as nx
import igraph as ig
import numpy as np

In [2]:
#get sample graph from networkx
n = 100_000
m = 3
G = nx.barabasi_albert_graph(n,m)

##add random weights to the graph
weighted_edge_list = []

for u,v in G.edges():
    weight = np.random.rand() *100
    weighted_edge_list.append((u,v,weight))
    #add weight to graph
    G[u][v]['weight'] = weight


print(f"number of nodes: {G.number_of_nodes()}")
print(f"number of edges: {G.number_of_edges()}")

number of nodes: 100000
number of edges: 299991


In [3]:
G_rust = netrustpy3.Graph.from_weighted_edge_list(weighted_edge_list)

print(f"Number of nodes: {G_rust.num_v()}")
print(f"Number of edges: {G_rust.num_e()}")

Number of nodes: 100000
Number of edges: 299991


In [88]:
node = G.nodes()

In [90]:
#created weighted igraph from weighted edge list
node = G.nodes()
G_igraph = ig.Graph(directed=False)
G_igraph.add_vertices(n)
G_igraph.add_edges([(u,v) for u,v in G.edges()])
G_igraph.es['weight'] = [G[u][v]['weight'] for u,v in G.edges()]

print(f"Number of nodes: {G_igraph.vcount()}")
print(f"Number of edges: {G_igraph.ecount()}")

Number of nodes: 100000
Number of edges: 299991


In [91]:
%%timeit
bfs_tree = nx.bfs_tree(G, 0)

764 ms ± 17.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [92]:
%%timeit
bfs_tree = netrustpy3.bfs(G_rust, 0)

100 ms ± 5.73 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [93]:
%%timeit
bfs_tree = ig.Graph.bfs(G_igraph, 0)

32.2 ms ± 1.4 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [95]:
#disjtrak
nodes = list(G.nodes())
shortest_path = nx.single_source_dijkstra(G, 0)
shortest_path_rust = netrustpy3.single_source_dijkstra(G_rust, 0)
shortest_path_igraph = G_igraph.distances(source = 0, target= nodes ,weights='weight', algorithm='dijkstra', mode="all")

In [96]:
node = 10
print(shortest_path[0][node])
print(shortest_path_rust[node][0])
print(shortest_path_igraph[0][node])

21.913286932798147
21.913288116455078
21.913286932798147


In [97]:
paths = G_igraph.get_shortest_paths(0, to=nodes, weights='weight', mode='ALL')

In [98]:
shortest_path[1][node]

[0, 2, 14517, 1760, 65155, 14, 10]

In [99]:
shortest_path_rust[node][1]

[2, 14517, 1760, 65155, 14, 10]

In [100]:
paths[node]

[0, 2, 14517, 1760, 65155, 14, 10]

In [101]:
%%timeit
shortest_path = nx.single_source_dijkstra(G, 0)

1.94 s ± 72.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [102]:
%%timeit
shortest_path_rust = netrustpy3.single_source_dijkstra(G_rust, 0)

416 ms ± 57 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [103]:
%%timeit
shortest_path_igraph = G_igraph.distances(source = 0, target= nodes ,weights='weight', algorithm='dijkstra', mode="all")

198 ms ± 12.5 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [106]:
bfs_tree = netrustpy3.bfs(G_rust, 0)

In [112]:
bfs_tree.nodes

AttributeError: 'builtins.BfsTree' object has no attribute 'nodes'

In [107]:
print(f"distance from node 0 to node 3: {bfs_tree[10000][1]}")

# get path from node 1 to node 3
path = []
node = 10000
while node != None:
    path.append(node)
    node = bfs_tree[node][0]

print(f"path from node 1 to node 3: {path[::-1]}")

TypeError: 'builtins.BfsTree' object is not subscriptable